# 探索型データ分析
Nishokaで取得したサンプルデータを利用しPythonライブラリによるデータ操作の基本を学ぶ。  
データ内容を整理、視覚化し機械学習を行うための準備をする

参考:Nishika・中古マンション価格予測 https://competition.nishika.com/competitions/mansion_pra/summary

In [4]:
# データ分析に必要なライブラリを読み込む
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
# 全てのサンプルCSVを読み込む
# CSVデータを配列で取得
files = glob.glob("workspace/train/*.csv")


In [6]:
# 取得したデータをread_csvで操作可能な値としてdata_listに配列として保存する
data_list = []
for file in files:
  data_list.append(pd.read_csv(file, index_col=0))

/tmp/ipykernel_11810/1387233916.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))
/tmp/ipykernel_11810/1387233916.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_list.append(pd.read_csv(file, index_col=0))


In [7]:
# 配列内のデータを全て１つに結合する
df = pd.concat(data_list)

In [8]:
# 行数と列数の表示
df.shape

(637351, 27)

In [9]:
# 欠損データ数とデータタイプの表示
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637351 entries, 1060685 to 47003572
Data columns (total 27 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   種類            637351 non-null  object 
 1   地域            0 non-null       float64
 2   市区町村コード       637351 non-null  int64  
 3   都道府県名         637351 non-null  object 
 4   市区町村名         637351 non-null  object 
 5   地区名           637060 non-null  object 
 6   最寄駅：名称        634732 non-null  object 
 7   最寄駅：距離（分）     614306 non-null  object 
 8   間取り           615609 non-null  object 
 9   面積（㎡）         637351 non-null  object 
 10  土地の形状         0 non-null       float64
 11  間口            0 non-null       float64
 12  延床面積（㎡）       0 non-null       float64
 13  建築年           619117 non-null  object 
 14  建物の構造         623423 non-null  object 
 15  用途            591214 non-null  object 
 16  今後の利用目的       275091 non-null  object 
 17  前面道路：方位       0 non-null       float64
 

In [10]:
# データが全てnullのカラムを削除したい

# 空のカラムをリストに入れたい
nonnull_list = []

# データフレームの全ての各カラムに対しループ処理
for col in df.columns:
  nonnull = df[col].count()
  if nonnull == 0:
    # データが0のカラムをリストに追加
    nonnull_list.append(col)

nonnull_list

['地域', '土地の形状', '間口', '延床面積（㎡）', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）']

In [12]:
# 上記の空のカラムを除外する
df = df.drop(nonnull_list, axis=1)

axisオプションはその処理を行に沿って実行するか、列に沿って実行するか指定する  
https://qiita.com/Phoeboooo/items/b464b7df3c64a33caf94

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637351 entries, 1060685 to 47003572
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   種類            637351 non-null  object 
 1   市区町村コード       637351 non-null  int64  
 2   都道府県名         637351 non-null  object 
 3   市区町村名         637351 non-null  object 
 4   地区名           637060 non-null  object 
 5   最寄駅：名称        634732 non-null  object 
 6   最寄駅：距離（分）     614306 non-null  object 
 7   間取り           615609 non-null  object 
 8   面積（㎡）         637351 non-null  object 
 9   建築年           619117 non-null  object 
 10  建物の構造         623423 non-null  object 
 11  用途            591214 non-null  object 
 12  今後の利用目的       275091 non-null  object 
 13  都市計画          618786 non-null  object 
 14  建ぺい率（％）       614848 non-null  float64
 15  容積率（％）        614848 non-null  float64
 16  取引時点          637351 non-null  object 
 17  改装            580431 non-null  object 
 

### 多重共線性  
重回帰モデルにおいて、説明変数の中に相関係数が高い組み合わせがあるモデルの予測や信頼性が下がる  
上記データのうち、相関の高い要素を除外したい

In [17]:
# 市区町村と市区町村コードは完全に一致したものではないか？

# value_countsを使用して、データの種類と数を比較する
df["市区町村コード"].value_counts()

13111    12269
13108    11591
13112    10747
13103    10580
13104    10055
         ...  
40604        1
40213        1
40212        1
40341        1
34309        1
Name: 市区町村コード, Length: 618, dtype: int64

In [18]:
df["市区町村名"].value_counts()

大田区       12269
江東区       11591
世田谷区      10747
港区        10580
新宿区       10055
          ...  
田川郡糸田町        1
行橋市           1
大川市           1
粕屋郡宇美町        1
安芸郡坂町         1
Name: 市区町村名, Length: 618, dtype: int64

市区町村コードと市区町村名において、データの種類と数が完全に一致している  
上記の結果は、市区町村名と対応する市区町村コードが618個あり、そのそれぞれの出現回数も一致していることを表す

In [19]:
# どちらか一方のカラム(今回は市区町村名)を削除する
df = df.drop("市区町村名", axis=1)

In [26]:
# 統計量の確認

# デフォルトでは数値型が分析対象となり、データ数、平均、四分位数など表示される
df.describe()

,市区町村コード,建ぺい率（％）,容積率（％）,取引価格（総額）_log
count,637351.000000,614848.000000,614848.000000,637351.000000
mean,18513.985300,67.601944,301.601876,7.217424
std,9596.722442,10.402295,148.105400,0.353935
min,1101.000000,30.000000,50.000000,2.653213
25%,13106.000000,60.000000,200.000000,7.000000
50%,14104.000000,60.000000,200.000000,7.255273
75%,27114.000000,80.000000,400.000000,7.447158
max,47213.000000,80.000000,1300.000000,9.934498


In [27]:
# astypeを使い分析対象を文字列型に指定すると、データ数、データ種類が表示される
df.astype(str).describe()

,種類,市区町村コード,都道府県名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
count,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351,637351
unique,1,618,47,12047,3832,35,64,114,77,13,23,5,17,7,18,58,3,10,380
top,中古マンション等,13111,東京都,本町,nan,6,３ＬＤＫ,70,平成3年,ＲＣ,住宅,nan,商業地域,60.0,200.0,2014年第１四半期,未改装,nan,7.176091259055681
freq,637351,12269,190524,5267,2619,50928,272449,83443,21982,431050,583722,362260,172628,349347,293388,13561,419268,619076,20996


上記の中でデータの種類が1つしかない物を除外する  

In [28]:
df["種類"].value_counts()

中古マンション等    637351
Name: 種類, dtype: int64

In [30]:
# 種類カラムを削除
df = df.drop("種類", axis=1)